# Xây dựng LLM

In [4]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.1 MB/s eta 0:00:00


## Xây dựng Database

In [5]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub



In [6]:
Loader = PyPDFLoader
FILE_PATH = './AIO-2024-All-Materials.pdf'
loader = Loader(FILE_PATH)
documents = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(documents)

print('Number of sub-documents: ', len(docs))
print(docs[0])

Number of sub-documents:  33
page_content='AI VIET NAM – AI COURSE 2024
Tutorial: Phát hiện đối tượng trong ảnh với
YOLOv10
Dinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và
Quang-Vinh Dinh
Ngày 20 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực
Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong
một tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm
giải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look
Once) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực
thi mà loại mô hình này mang lại.
Hình 1: Logo của mô hình YOLO. Ảnh: link.
Thời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)
đã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object
Detection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu suất vượ

In [9]:
embedding = HuggingFaceEmbeddings()

In [10]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embedding)
retriever = vector_db.as_retriever()

result = retriever.invoke('What is YOLO?')

print('Number of relevant documents: ', len(result))

Number of relevant documents:  4


## Khởi tạo mô hình ngôn ngữ lớn

1. Khai báo một số cài đặt cần thiết cho mô hình vì mô hình ngôn ngữ lớn yêu cầu tài nguyên về phần cứng khá lớn

In [11]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

2. Khởi tạo mô hình và tokenizer

In [12]:
MODEL_NAME = 'lmsys/vicuna-7b-v1.5'

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

3. Tích hợp tokenizer và model thành một pipeline để tiện sử dụng

In [13]:
model_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map='auto'
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline
)

## Chạy chương trình

In [14]:
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
  {"context":retriever | format_docs, "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)


In [15]:
USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer: ')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tạo bằng cách tải về trọng số (file.pt) từ GitHub và khởi tạo mô hình bằng cách sử dụng thư viện ultralytics.


# Xây dựng giao diện chat

Sử dụng lại thư viện đã cài đặt ở phân trên cùng với các hàm và instance đã cài đặt

In [16]:
!pip install -q langchain-openai==0.1.9
!pip install -q chainlit==1.1.304
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import chainlit as cl
import torch

from chainlit.types import AskFileResponse

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [21]:
#Xây dựng hàm xử lý file input đầu vào
def process_file(file: AskFileResponse):
  if file.type == 'text/plain':
    Loader = TextLoader
  elif file.type == 'application/pdf':
    Loader = PyPDFLoader
  loader = Loader(file.path)
  documents = loader.load()
  docs = text_splitter.split_documents(documents)
  for i, doc in enumerate(docs):
    doc.metadata['source'] = f"source_{i}"
  return docs

In [22]:
#Xây dựng hàm khởi tạo Chroma database
def get_vector_db(file: AskFileResponse):
  docs = process_file(file)
  cl.user_session.set('docs', docs)
  vector_db = Chroma.from_documents(documents=docs,
                                    embedding=embedding)
  return vector_db

In [23]:
# Khởi tạo mô hình ngôn ngữ lớn:
def get_huggingface_llm(model_name: str = 'lmsys/vicuna-7b-v1.5',
                        max_new_token: int = 512):
  nf4_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=nf4_config,
      low_cpu_mem_usage=True
  )

  tokenizer = AutoTokenizer.from_pretrained(model_name)

  model_pipeline = pipeline(
      'text-generation',
      model=model,
      tokenizer=tokenizer,
      max_new_tokens=max_new_token,
      pad_token_id=tokenizer.eos_token_id,
      device_map='auto'
  )

  llm = HuggingFacePipeline(
      pipeline=model_pipeline
  )
  return llm

LLM = get_huggingface_llm()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [24]:
#Khởi tạo welcome mesage:
welcome_message = """ Welcome to the PDF QA! To get started :
Upload a PDF or text file
Ask a question about the file
"""

In [26]:
#Khởi tạo hàm on_chat_start:
@cl.on_chat_start
async def on_chat_start():
  files = None
  while files is None:
    file = await cl.AskFileMessage(
        content=welcome_message,
        accept=['text/plain','application/pdf'],
        max_size_mb=20,
        timeout=180
    ).send()
  file = files[0]

  msg = cl.Message(content=f"Processing '{file.name}'...",
                   disable_feedback=True)
  await msg.send()

  vector_db = await cl.make_async(get_vector_db)(file)

  message_history = ChatMessageHistory()
  memory = ConversationBufferMemory(
      memory_key='chat_history',
      output_ket='answer',
      chat_memory=message_history,
      return_messages=True
  )
  retriecer = vector_db.as_retriever(search_type='mmr',
                                     search_kwargs={'k': 3})

  chain = ConversationalRetrievalChain.from_llm(
      llm=LLm,
      chain_type='stuff',
      retriever=retriever,
      memory=memory,
      return_source_documents=True
  )

  msg.content = f"'{file.name}' processed. You can now ask question"
  await msg.update()

  cl.user_session.set('chain', chain)


In [27]:
#Khởi tạo hàm on_message:
@cl.on_message
async def on_message(message: cl.Message):
  chain = cl.user_session.get('chain')
  cb = cl.AsyncLangchainCallbackHandler()
  res = await chain.ainvoke(message.content, callbacks=[cb])
  answer = res['answer']
  source_documents = res['source_documents']
  text_elements = []

  if source_documents:
    for source_idx, source_doc in enumerate(source_documents):
      source_name = f'source_{source_idx}'
      text_elements.append(
          cl.Text(content=source_doc.page_content,
                  name=source_name)
      )
    source_names = [text_el.name for text_el in text_elements]

    if source_names:
      answer += f"\nSources: {', '.join(source_names)}"
    else:
      answer += "\nNo sources found"

  await cl.Message(content=answer, elements=text_elements).send()

In [34]:
!chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt &

In [35]:
import urllib
print("Password/Endpoints IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip('\n'))

Password/Endpoints IP for localtunnel is: 34.125.209.69


In [ ]:
!lt --port 8000 --subdomain aivn-simple-rag

your url is: https://aivn-simple-rag.loca.lt
